In [ ]:
!pip install -q -U pip
!pip install -q torch==2.1.0a0+32f93b1
!pip install -q transformers==4.43.3
!pip install -q peft==0.12.0
!pip install -q jinja2==3.1.4
!pip install -q bitsandbytes==0.43.0
!pip install -q datasets==2.20.0
!pip install -q multiprocess==0.70.16
!pip install -q accelerate==0.33.0
!pip install -q trl==0.9.6

In [ ]:
!pip list

In [ ]:
# 패키지 경로 환경변수 설정
!export LD_LIBRARY_PATH="~/.local/lib/python3.10/site-packages/:$LD_LIBRARY_PATH"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer, EarlyStoppingCallback
import os 
import re
import torch
import pandas as pd
from datasets import Dataset
from trl import SFTTrainer

In [ ]:
import huggingface_hub
huggingface_hub.login('')

In [ ]:
# # 모델 다운로드
# from huggingface_hub import snapshot_download
# snapshot_download(repo_id="MLP-KTLim/llama-3-Korean-Bllossom-8B", local_dir = "./models_llama-3-Korean-Bllossom-8B")

In [ ]:
# 로컬 디렉토리에서 토크나이저와 모델 로드
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
local_dir = "./models_llama-3-Korean-Bllossom-8B"
tokenizer = AutoTokenizer.from_pretrained(local_dir)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=False,
    bnb_8bit_quant_type="int8", 
    bnb_8bit_compute_dtype=torch.float16 
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    local_dir,
    quantization_config=bnb_config,  #양자화 적용
    )

In [ ]:
# 학습가능 파라미터 개수와 비율 표시
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(model)

In [ ]:
# Lora 학습용 변수 설정. 모델 구조에 따라 target_modules는 바꿔줘야 한다. 
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
        "mlp.gate_proj",
        "mlp.up_proj",
        "mlp.down_proj"
    ],
    lora_dropout=0.01,
    bias="none",
    task_type="CAUSAL_LM"
)

# 모델에 lora를 적용한다. 
model = get_peft_model(model, lora_config)

print_trainable_parameters(model)

In [ ]:
# 학습 가중치 불러오기
model.load_state_dict(torch.load("./con_trained"))

In [ ]:
# 모델이 사용중인 device 확인
#print(f"Model is using device: {next(model.parameters()).device}")

In [ ]:
# 모델 구조 확인
# for name, module in model.named_modules():
#     print(name, type(module))

# 데이터셋 만들기

In [ ]:
system_prompt = """
당신은 OKR 문장 평가 전문가입니다.
평가기준은 '연결성'입니다. 평가기준에 대한 설명을 보고, 이를 엄격히 준수해서 평가합니다.
평가 기준에 따라 평가할 때, 상위목표, 기업명, 업종, 조직명을 반드시 참고해 주십시오.

# 평가기준
## 평가기준에 대한 설명
연결성은 다음과 같은 두 가지 기준으로 평가합니다.
기준 1. 상위목표를 달성하는 데 핵심결과가 기여하는가?
기준 2. 핵심결과 문장이 나타내는 바가 구체적이고 명확한가?
## 평가기준에 따른 점수
평가기준에 대한 설명을 반드시 엄격히 준수합니다. 그리고 평가기준을 만족하는 정도에 따라 점수를 다르게 주세요.
4~5점: 기준 1과 기준 2를 모두 만족한다.
3~3.5점: 기준 1은 만족하지만, 기준 2는 만족하지 못한다.
2~2.5점: 기준2는 만족하지만, 기준 1은 만족하지 못한다.
1~1.5점: 기준 1과 기준 2를 모두 만족하지 못한다.

# 출력
출력은 반드시 아래와 같은 형식을 엄격히 지켜주세요.

점수: [1부터 5사이의 점수, 소수점 단위 0.5]
이유: [점수를 준 이유]
"""

input_template = """
상위목표 = {upper_objective}
핵심결과 = {input_sentence}
"""

output_template = """
점수: {connectivity}
이유: {connectivity_description}
"""

In [ ]:
# 데이터셋 불러오기
df = pd.read_csv('revision_240806_dataset_plus.csv')

dfKeyresult = df[df["type"] == "Key Result"]  # keyResult행만 추출
dfKeyresult = dfKeyresult[['row_num', 'type', 'input_sentence', 'upper_objective', 'company', 'field', 'team', 'connectivity', 'connectivity_description', 'predict_connectivity', 'predict_connectivity_description']]  #특정 열만 추출
dfKeyresult = dfKeyresult.dropna(subset=["input_sentence", "upper_objective", "connectivity", "connectivity_description"])  #결측값 있는 행 삭제

In [ ]:
print(dfKeyresult)

In [ ]:
# 데이터를 train, test 2가지 용도로 분할. random_state로 랜덤 시드를 지정한다. 
from sklearn.model_selection import train_test_split
     
train_df, test_df = train_test_split(dfKeyresult, test_size=0.1, random_state=100)

#print(train_df)

In [ ]:
# 원시 데이터프레임에 템플릿 적용하고 리스트로 변환
def data2list(df):
    input_conversations= []
    for index, row in df.iterrows():
        input_text = input_template.format(
            upper_objective=row["upper_objective"],
            input_sentence=row["input_sentence"]
        )

        output_text = output_template.format(
            connectivity=row["connectivity"],
            connectivity_description=row["connectivity_description"]
        )
        
        input_conversation = { 'messages' : [{"role": "system", "content": f"{system_prompt.strip()}"},
                                            {"role": "user", "content": f"{input_text.strip()}"},
                                            {"role": "assistant", "content": f"{output_text.strip()}"}]
        }
        
        input_conversation = {"text": tokenizer.apply_chat_template(input_conversation['messages'], tokenize=False)}
        input_conversations.append(input_conversation)
    
    return input_conversations

train_dataset = data2list(train_df)
train_dataset = Dataset.from_list(train_dataset)
test_dataset = data2list(test_df)
test_dataset = Dataset.from_list(test_dataset)

# 모델 학습

In [ ]:
model.train()
print("train mode on")

In [ ]:
# 훈련 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=30,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_strategy = "epoch",
    load_best_model_at_end=True,
    save_strategy="epoch",
)

early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

In [ ]:
# Trainer 설정
trainer = SFTTrainer(
    model=model,
    args=training_args,
    dataset_text_field="text",
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #callbacks=[early_stopping]
)

# 모델 학습
trainer.train()

In [ ]:
import mlflow
mlflow.delete_run('')

In [ ]:
# 모델 저장
torch.save(model.state_dict(), "./con_trained")

# model.save_pretrained("./trained_model")
# tokenizer.save_pretrained("./trained_model")

# 모델 테스트

In [ ]:
def generate(prompt):
    messages = [
        {"role": "system", "content": f"{system_prompt}"},
        {"role": "user", "content": f"{prompt}"}
        ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    attention_mask = (input_ids != tokenizer.pad_token_id).long()
    
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,  # Add attention mask
        max_new_tokens=2048,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,  # Set pad token ID to eos token ID
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    torch.cuda.empty_cache()
    return tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

In [ ]:
def extract_data(text):
    # 정규 표현식을 사용하여 상위목표, 핵심결과, 점수, 이유를 추출
    upper_objective_match = re.search(r"상위목표\s*=\s*(.*)", text)
    input_sentence_match = re.search(r"핵심결과\s*=\s*(.*)", text)
    score_match = re.findall(r"점수:\s*([\d.]+)", text)
    reason_match = re.findall(r"이유:\s*(.*)", text)

    # 추출된 결과
    if upper_objective_match:
        upper_objective = upper_objective_match.group(1).strip()

    if input_sentence_match:
        input_sentence = input_sentence_match.group(1).strip()

    if score_match:
        if len(score_match) > 1:
            score = score_match[1].strip()
        else:
            score = score_match[0].strip()

    if reason_match:
        if len(reason_match) > 1:
            reason = reason_match[1].strip()
        else:
            reason = reason_match[0].strip()
    
    return upper_objective, input_sentence, score, reason

In [ ]:
# 생성 결과 저장용
def extract_data2(text):
    score_match = re.findall(r"점수:\s*([\d.]+)", text)
    description_match = re.findall(r"이유:\s*(.*)", text)

    predict_score = score_match[0].strip() if score_match else None
    predict_description = description_match[0].strip() if description_match else None

    return predict_score, predict_description

In [ ]:
# train 데이터 평가
print("[krEV]")
#start_index = int(input("enter start index: "))
for index, row in dfKeyresult.iterrows():
    if pd.isna(dfKeyresult.loc[index, 'predict_connectivity']):
        print(f"idx : {index}")

        upper_objective, input_sentence, connectivity, connectivity_description = row[['upper_objective', 'input_sentence', 'connectivity', 'connectivity_description']]
        print(f"상위목표: {upper_objective}")
        print(f"핵심결과: {input_sentence}")
        print("<전문가>")
        print(f"_점수: {connectivity}")
        print(f"_이유: {connectivity_description}")

        input_text = input_template.format(upper_objective = upper_objective, input_sentence = input_sentence)
        #print(text)
        print()
        print("<AI>")

        predict = generate(input_text)
        print(predict)
        print(type(predict))
        predict_score, predict_description = extract_data2(predict)
        print(f"predict_score: {predict_score}")
        print(f"predict_description: {predict_description}")

        dfKeyresult.loc[index, ['predict_connectivity', 'predict_connectivity_description']] = predict_score, predict_description

        print('='*100)
        print('='*100)
        print() 

In [ ]:
print(dfKeyresult)

# 모델 간단 테스트

In [ ]:
system_prompt = """
당신은 OKR 문장 평가 전문가입니다. 
핵심결과 달성이 상위목표 실현에 얼마나 연결되어 있는지 점수(1-5)와 평가 이유를 구체적으로 제공하십시오.
점수는 1은 거의 기여하지 않음, 5는 매우 기여함을 의미합니다. 
기업명, 업종, 조직명을 참고해 주십시오. 
반드시 아래의 출력 형식을 지키십시오.

점수: [1-5 범위의 숫자]
이유: [점수를 준 이유]
"""

ex_text = """
<입력>
상위목표: 보다 많은 사람들이 서비스를 이용한다(고객의 양적 창출)
핵심결과: 신규 사용자 수를 10만 명에서 100만 명으로 늘린다.
</입력>
"""

In [ ]:
generate(system_prompt, ex_text)

In [ ]:
torch.cuda.empty_cache()